# 01 - Create datasets for qualitative analysis with M0

In [6]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1" #Numéro GPU
print(os.environ["CUDA_VISIBLE_DEVICES"])

1


In [2]:
import os, sys
from pathlib import Path

ENV_IS_GOOGLE_COLAB = True if 'google.colab' in str(get_ipython()) else False
os.environ["ENV_IS_GOOGLE_COLAB"] = str(ENV_IS_GOOGLE_COLAB)

if ENV_IS_GOOGLE_COLAB:
  from google.colab import drive
  mountpoint = Path("/content/drive")
  drive.mount(str(mountpoint)) # Mount gdrive to BASE
  base = mountpoint / "MyDrive/article_icdar_2023" # Adapt this to your situation
  sys.path.append(str(base)) # Add BASE to Python Path
  BASE = Path(base).resolve() # Make BASE absolute
  DATASETS =  BASE / "dataset"
else:
  BASE = Path(os.path.dirname(os.path.realpath("__file__"))).resolve() # If not on GColab, BASE will be the directory of this notebook
  DATASETS = Path('/lrde/home2/stual/code_ICDAR_review/dataset').resolve()
  OUT_BASE = Path('/work/stual/res_ICDAR/method_0').resolve()

print(sys.path)
print(BASE)
print(DATASETS)
print(OUT_BASE)

['/lrde/home2/stual/code_ICDAR_review/src/t1_dataset_tools', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/lrde/home2/stual/.local/lib/python3.10/site-packages', '/usr/lib/python3.10/site-packages']
/lrde/home2/stual/code_ICDAR_review/src/t1_dataset_tools
/lrde/home2/stual/code_ICDAR_review/dataset
/work/stual/res_ICDAR/method_0


In [3]:
CREATE_REF_IO = True
CREATE_PERO_IO = True

Set value of **MODEL_NAME** to *camembert_ner* or *pretrained_camembert_ner*

In [4]:
MODEL_NAME = 'camembert_ner'
#pretrained_camembert_ner

In [9]:
import glob
import re
import json
import pandas as pd
import config
import os
from pathlib import Path

config.SPLIT_SEED = 42 # Random seed used in train/dev/test. Do not change it if you want to recreate the paper results.
config.DEBUG = False # If true, text versions of the spacy datasets will be saved along with the .spacy files.

ModuleNotFoundError: No module named 'config'

## 01.1 Reference data

In [6]:
#Sanity check
DATA = DATASETS / "/41-ner_ref_from_pero"
f = open(f"{DATA}/all_brat.json")
data = json.load(f)
f.close()

In [7]:
gold_reference = pd.DataFrame.from_records(data)
gold_reference

,id,box,book,page,valid_box,text_ocr,ner_xml,nested_ner_xml,brat
0,290,"[125.49036875669651, 357.68003651377927, 386.0...",Bottin1_1820,107,True,"Dufant (Victor), libraire, r. du Gros-Che-\nne...","<PER>Dufant (Victor)</PER>, <ACT>libraire</ACT...","<PER>Dufant (Victor)</PER>, <ACT>libraire</ACT...","[Dufant (Victor), libraire, r. du Gros-Che- ne..."
1,292,"[127.0, 398.61236450520784, 387.0, 40.38763549...",Bottin1_1820,107,True,"Dufay, essayeur du commerce, place Dau-\nphine...","<PER>Dufay</PER>, <ACT>essayeur du commerce</A...","<PER>Dufay</PER>, <ACT>essayeur du commerce</A...","[Dufay, essayeur du commerce, place Dau- phine..."
2,294,"[124.46245769617323, 436.6867055467269, 388.52...",Bottin1_1820,107,True,"Dulay, chandronnier, r. du Pont- aux\nChoux, 1...","<PER>Dulay</PER>, <ACT>chandronnier</ACT>, <LO...","<PER>Dulay</PER>, <ACT>chandronnier</ACT>, <SP...","[Dulay, chandronnier, r. du Pont- aux Choux, 1..."
3,298,"[122.96782088243424, 476.6769851466119, 388.0,...",Bottin1_1820,107,True,"Dufay (V.e), grenetière, r. du Faub.-S.\nDenis...","<PER>Dufay (V.e)</PER>, <ACT>grenetière</ACT>,...","<PER>Dufay (V.e)</PER>, <ACT>grenetière</ACT>,...","[Dufay (V.e), grenetière, r. du Faub.-S. Denis..."
4,312,"[126.68533584543587, 648.8951119484786, 393, 29]",Bottin1_1820,107,True,"Dufeu, charcutier, r. Montmartre, 89. 318","<PER>Dufeu</PER>, <ACT>charcutier</ACT>, <LOC>...","<PER>Dufeu</PER>, <ACT>charcutier</ACT>, <SPAT...","[Dufeu, charcutier, r. Montmartre, 89. 318, [T..."
...,...,...,...,...,...,...,...,...,...
8440,343,"[269.0, 1266.0, 405.0, 26.0]",Notables_communaux_seine_1801,144,True,"Lamarche, géographe , rue du foin.","<PER>Lamarche</PER>, <ACT>géographe</ACT> , <L...","<PER>Lamarche</PER>, <ACT>géographe</ACT> , <S...","[Lamarche, géographe , rue du foin., [T0, PER,..."
8441,345,"[267.0, 1293.0, 627.0, 26.0]",Notables_communaux_seine_1801,144,True,"Lamarck, membre de l'institut, quatrième munic...","<PER>Lamarck</PER>, <ACT>membre de l&apos;inst...","<PER>Lamarck</PER>, <ACT>membre de l'institut<...","[Lamarck, membre de l'institut, quatrième muni..."
8442,347,"[267, 1321, 478, 26]",Notables_communaux_seine_1801,144,True,"Lamare, notaire, rue du faubourg honoré.","<PER>Lamare</PER>, <ACT>notaire</ACT>, <LOC>ru...","<PER>Lamare</PER>, <ACT>notaire</ACT>, <SPAT><...","[Lamare, notaire, rue du faubourg honoré., [T0..."
8443,349,"[267, 1349, 383, 25]",Notables_communaux_seine_1801,144,True,"Lamarre , carrier, rue mouffetard.","<PER>Lamarre</PER> , <ACT>carrier</ACT>, <LOC>...","<PER>Lamarre</PER> , <ACT>carrier</ACT>, <SPAT...","[Lamarre , carrier, rue mouffetard., [T0, PER,..."


In [8]:
gold_reference_valid = gold_reference[['brat','book']]
gold_reference_valid

,brat,book
0,"[Dufant (Victor), libraire, r. du Gros-Che- ne...",Bottin1_1820
1,"[Dufay, essayeur du commerce, place Dau- phine...",Bottin1_1820
2,"[Dulay, chandronnier, r. du Pont- aux Choux, 1...",Bottin1_1820
3,"[Dufay (V.e), grenetière, r. du Faub.-S. Denis...",Bottin1_1820
4,"[Dufeu, charcutier, r. Montmartre, 89. 318, [T...",Bottin1_1820
...,...,...
8440,"[Lamarche, géographe , rue du foin., [T0, PER,...",Notables_communaux_seine_1801
8441,"[Lamarck, membre de l'institut, quatrième muni...",Notables_communaux_seine_1801
8442,"[Lamare, notaire, rue du faubourg honoré., [T0...",Notables_communaux_seine_1801
8443,"[Lamarre , carrier, rue mouffetard., [T0, PER,...",Notables_communaux_seine_1801


In [9]:
from dataset_util import train_dev_test_split, unwrap # Local imports
from sklearn.model_selection import train_test_split

# CONSTANTS
MIN_TRAINSET_SIZE = 30

 # Split 72/8/20% w. stratified sampling on directories names
train, dev, test = train_dev_test_split(gold_reference_valid.to_numpy())

# Iteratively split the trainset in half to create smaller trainsets
exp1_trainsets = [train]
t_len = len(train)

while True:
    try:
        current = exp1_trainsets[-1]
        _, groups = unwrap(current)
        smaller, rest = train_test_split(
            current,
            train_size=0.5,
            shuffle=True,
            random_state=config.SPLIT_SEED,
            stratify=groups,
        )
        t_len = len(rest)
        if t_len < MIN_TRAINSET_SIZE:
            break
        exp1_trainsets.append(smaller)

    except ValueError:
        # Stop now if we encounter the error "The least populated class in y has only 1 member".
        break

[len(s) for s in exp1_trainsets] # Should be 

[6084, 3042, 1521, 760, 380, 190, 95, 47]

In [10]:
# Sanity checks

# Dev set should contain 676 examples
assert len(dev) == 676

# Test set should contain 1685 examples
assert len(test) == 1685

# Lenghts of exp1_trainsets should be fixed
assert sorted([len(s) for s in exp1_trainsets] ) == sorted([6084, 3042, 1521, 760, 380, 190, 95, 47])

In [11]:
if CREATE_REF_IO:
    from util_io_m0 import save_dataset_io
    
    RES = OUT_BASE / f"m0-experiment_1_prepared_dataset_ref_io_{MODEL_NAME}_v2"
    if not os.path.exists(RES):
        os.makedirs(RES)
    NAMES = ["train","dev","test"]
    
    for train in exp1_trainsets:
        datasets = [train, dev, test]
        save_dataset_io(RES, datasets, ["train","dev","test"], suffix=len(train))

2023-02-22 09:02:33.254052: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-22 09:02:34.830697: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-02-22 09:02:34.830796: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-02-22 09:02:34.830805: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Saving the dataset (0/1 shards):   0%|          | 0/6084 [00:00<?, ? examples/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Saving the dataset (0/1 shards):   0%|          | 0/47 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/676 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1685 [00:00<?, ? examples/s]

## 01.2 Noisy data

In [12]:
#Sanity check
PATH = DATASETS / "61-ner_align_pero_brat/"
f = open(f"{PATH}/all_brat.json")
data = json.load(f)
f.close()

In [13]:
gold_reference = pd.DataFrame.from_records(data)
gold_reference

,id,book,page,text_ocr,nested_ner_xml,has_valid_ner_xml,brat
0,286,Bottin1_1820,107,"Dufau et Clémendot, pharmaciens, r. de la\nChä...","<PER>Dufau et Clémendot</PER>, <ACT>pharmacien...",False,"[Dufau et Clémendot, pharmaciens, r. de la Chä..."
1,290,Bottin1_1820,107,"☞\n\nT\nDufant (Victor), libraire, r. du Gros-...","☞ T <PER>Dufant (Victor)</PER>, <ACT>libraire...",True,"[☞ T Dufant (Victor), libraire, r. du Gros-Ch..."
2,292,Bottin1_1820,107,"Dutay, essayeur du commerce, place Dau-\n\n-\n...","<PER>Dutay</PER>, <ACT>essayeur du commerce</A...",True,"[Dutay, essayeur du commerce, place Dau- - pí..."
3,294,Bottin1_1820,107,"Dulay, chandronnier, r. du Pont- aux-\nChuux, ...","<PER>Dulay</PER>, <ACT>chandronnier</ACT>, <SP...",True,"[Dulay, chandronnier, r. du Pont- aux- Chuux, ..."
4,298,Bottin1_1820,107,"Dufay (V.e), grenetière, r. du Fauh.-S.\nDenis...","<PER>Dufay (V.e)</PER>, <ACT>grenetière</ACT>,...",True,"[Dufay (V.e), grenetière, r. du Fauh.-S. Denis..."
...,...,...,...,...,...,...,...
8760,343,Notables_communaux_seine_1801,144,"Lamarche, geographe , rue du foin.","<PER>Lamarche</PER>, <ACT>geographe</ACT> , <S...",True,"[Lamarche, geographe , rue du foin., [T0, PER,..."
8761,345,Notables_communaux_seine_1801,144,"Lamarck, membre de l'institut, quatrieme munic...","<PER>Lamarck</PER>, <ACT>membre de l&apos;inst...",True,"[Lamarck, membre de l&apos;institut, quatrieme..."
8762,347,Notables_communaux_seine_1801,144,"Lamare , notaire, rue du faubourg honore.","<PER>Lamare</PER> , <ACT>notaire</ACT>, <SPAT>...",True,"[Lamare , notaire, rue du faubourg honore., [T..."
8763,349,Notables_communaux_seine_1801,144,"Lamarre, carrier, rue mouffetard.","<PER>Lamarre</PER>, <ACT>carrier</ACT>, <SPAT>...",True,"[Lamarre, carrier, rue mouffetard., [T0, PER, ..."


In [14]:
gold_reference_valid = gold_reference.loc[gold_reference['has_valid_ner_xml'] == True]
gold_reference_valid = gold_reference_valid[['brat','book']]
gold_reference_valid

,brat,book
1,"[☞ T Dufant (Victor), libraire, r. du Gros-Ch...",Bottin1_1820
2,"[Dutay, essayeur du commerce, place Dau- - pí...",Bottin1_1820
3,"[Dulay, chandronnier, r. du Pont- aux- Chuux, ...",Bottin1_1820
4,"[Dufay (V.e), grenetière, r. du Fauh.-S. Denis...",Bottin1_1820
5,"[Y ☞ Dnten,charentier, 1. Montmartre, 89. 318,...",Bottin1_1820
...,...,...
8760,"[Lamarche, geographe , rue du foin., [T0, PER,...",Notables_communaux_seine_1801
8761,"[Lamarck, membre de l&apos;institut, quatrieme...",Notables_communaux_seine_1801
8762,"[Lamare , notaire, rue du faubourg honore., [T...",Notables_communaux_seine_1801
8763,"[Lamarre, carrier, rue mouffetard., [T0, PER, ...",Notables_communaux_seine_1801


### Create subsets

In [15]:
from dataset_util import train_dev_test_split, unwrap # Local imports
from sklearn.model_selection import train_test_split

# CONSTANTS
MIN_TRAINSET_SIZE = 30

 # Split 72/8/20% w. stratified sampling on directories names
train, dev, test = train_dev_test_split(gold_reference_valid.to_numpy())

# Iteratively split the trainset in half to create smaller trainsets
exp1_trainsets = [train]
t_len = len(train)

while True:
    try:
        current = exp1_trainsets[-1]
        _, groups = unwrap(current)
        smaller, rest = train_test_split(
            current,
            train_size=0.5,
            shuffle=True,
            random_state=config.SPLIT_SEED,
            stratify=groups,
        )
        t_len = len(rest)
        if t_len < MIN_TRAINSET_SIZE:
            break
        exp1_trainsets.append(smaller)

    except ValueError:
        # Stop now if we encounter the error "The least populated class in y has only 1 member".
        break

[len(s) for s in exp1_trainsets] # Should be 

[6084, 3042, 1521, 760, 380, 190, 95, 47]

In [16]:
# Sanity checks

# Dev set should contain 676 examples
assert len(dev) == 676

# Test set should contain 1685 examples
assert len(test) == 1685

# Lenghts of exp1_trainsets should be fixed
assert sorted([len(s) for s in exp1_trainsets] ) == sorted([6084, 3042, 1521, 760, 380, 190, 95, 47])

In [17]:
if CREATE_PERO_IO:
    from util_io_m0 import save_dataset_io
    
    RES = OUT_BASE / f"m0-experiment_2_prepared_dataset_pero_ocr_io_{MODEL_NAME}_v2"
    if not os.path.exists(RES):
        os.makedirs(RES)
    NAMES = ["train","dev","test"]
    
    for train in exp1_trainsets:
        datasets = [train, dev, test]
        save_dataset_io(RES, datasets, ["train","dev","test"], suffix=len(train))

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Saving the dataset (0/1 shards):   0%|          | 0/6084 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/676 [00:00<?, ? examples/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

Saving the dataset (0/1 shards):   0%|          | 0/47 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/676 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1685 [00:00<?, ? examples/s]

In [18]:
from datasets import load_from_disk
set_ = load_from_disk(OUT_BASE / f"m0-experiment_2_prepared_dataset_pero_ocr_io_{MODEL_NAME}_v2/huggingface_6084")

/lrde/home2/stual/.venv/python_3_9/lib/python3.10/site-packages/datasets/dataset_dict.py:1241: FutureWarning: 'fs' was is deprecated in favor of 'storage_options' in version 2.8.0 and will be removed in 3.0.0.
You can remove this warning by passing 'storage_options=fs.storage_options' instead.
  warnings.warn(


In [19]:
set_["test"][0]

{'tokens': ['▁Au',
  'bert',
  'in',
  ',',
  '▁professeur',
  '▁au',
  '▁lycée',
  '▁Louis',
  '-',
  'le',
  '-',
  'Grand',
  ',',
  '▁Saint',
  's',
  '-',
  'P',
  'ères',
  ',',
  '▁7',
  '.'],
 'ner_tags': ['I-PER',
  'I-PER',
  'I-PER',
  'O',
  'I-ACT',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'I-LOC',
  'I-LOC',
  'I-LOC',
  'I-LOC',
  'I-LOC',
  'O',
  'I-CARDINAL',
  'O'],
 'input_ids': [5,
  277,
  21,
  2872,
  378,
  21,
  7,
  2136,
  36,
  3288,
  2027,
  67,
  16,
  67,
  1224,
  21,
  7,
  377,
  52,
  67,
  275,
  21,
  4487,
  21,
  7,
  333,
  21,
  9,
  6],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [-100,
  1,
  1,
  -100,
  1,
  0,
  -100,
  3,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -100,
  4,
  4,
  4,
  4,
  4,
  -100,
  0,
  -100,
  5,
  0,
  -100,
  -100]}